小住真央のデモノートです。コードは[こちら](https://github.com/m21kosumi/MyMatching.jl/blob/master/src/MyMatching.jl)

In [1]:
using MyMatching

testを実行します。

In [2]:
Pkg.test("MyMatching")

INFO: Testing MyMatching


Test Summary:               | Pass  Total
Testing deferred acceptance |   18     18


INFO: MyMatching tests passed


ShingakuMatchingを用いて進学選択シミュレーションを行います。

In [3]:
using ShingakuMatching

departments = get_departments()

num_students = 300 #number of students
students = get_students(num_students)

prop_prefs, resp_prefs, caps = get_random_prefs(students, departments)

(Array{Int64,1}[[23, 6, 56, 102, 77, 83, 123, 45, 51, 129  …  35, 75, 145, 148, 19, 116, 9, 84, 25, 27], [95, 89, 27, 54, 29, 126, 58, 128, 101, 118  …  151, 4, 31, 139, 49, 98, 91, 132, 102, 90], [100, 102, 76, 26, 34, 124, 56, 122, 58, 54  …  62, 36, 43, 20, 21, 40, 27, 138, 86, 123], [113, 145, 107, 37, 47, 97, 60, 142, 6, 79  …  85, 56, 105, 146, 25, 126, 136, 13, 118, 62], [18, 86, 102, 22, 14, 64, 10, 46, 126, 37  …  123, 47, 43, 27, 44, 60, 36, 61, 33, 30], [127, 132, 62, 15, 126, 58, 106, 91, 51, 134  …  6, 27, 86, 39, 33, 43, 49, 146, 29, 13], [142, 140, 111, 131, 54, 114, 41, 23, 35, 144  …  119, 81, 13, 51, 67, 139, 39, 125, 120, 109], [24, 41, 49, 42, 72, 56, 11, 74, 65, 112  …  66, 102, 22, 51, 62, 37, 52, 23, 38, 141], [45, 19, 34, 100, 60, 74, 64, 84, 18, 122  …  112, 50, 65, 49, 58, 121, 70, 48, 47, 124], [2, 86, 74, 53, 84, 124, 15, 123, 120, 97  …  19, 7, 138, 21, 17, 102, 47, 39, 121, 143]  …  [100, 31, 121, 3, 49, 51, 16, 86, 55, 40  …  22, 46, 76, 45, 25, 61, 13, 2

In [4]:
prop_matched, resp_matched, indptr = my_deferred_acceptance(prop_prefs, resp_prefs, caps)

([102, 95, 100, 113, 18, 127, 142, 24, 45, 84  …  121, 122, 113, 78, 56, 12, 1, 130, 58, 16], [297, 47, 179, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 119, 120, 121, 122, 123, 125, 206, 209, 227  …  881, 884, 890, 892, 898, 906, 930, 963, 971, 977])

以下では、どの生徒ともマッチングしなかった学科を探します。

In [9]:
r = 1
while r <= length(resp_prefs)
    if (resp_matched[indptr[r]:indptr[r+1]-1] == [0])
        println("resp $r ""got no student.")
    end
    r += 1
end

resp 4 got no student.
resp 5 got no student.
resp 22 got no student.
resp 32 got no student.


実際に上記の学科のresp_matchedが0になっているかどうかを確認してみます。

In [10]:
j = 4
resp_matched[indptr[j]:indptr[j+1]-1]

1-element Array{Int64,1}:
 0

In [11]:
j = 5
resp_matched[indptr[j]:indptr[j+1]-1]

1-element Array{Int64,1}:
 0

In [12]:
j = 22
resp_matched[indptr[j]:indptr[j+1]-1]

1-element Array{Int64,1}:
 0

In [13]:
j = 32
resp_matched[indptr[j]:indptr[j+1]-1]

1-element Array{Int64,1}:
 0

次に、各学科が受け入れた学生の数を調べます。

In [39]:
resp_num = Vector{Int}(length(resp_prefs))
for u in 1:length(resp_prefs)
    u_matched = resp_matched[indptr[u]:indptr[u+1]-1]
    stu_num = 1
    while !(u_matched[stu_num] == 0)
        stu_num += 1
        if stu_num > length(u_matched)
            break
        end
    end
    resp_num[u] = stu_num -1
end

resp_num

151-element Array{Int64,1}:
 3
 1
 1
 0
 0
 2
 3
 3
 6
 0
 3
 1
 2
 ⋮
 2
 1
 5
 3
 3
 0
 2
 0
 6
 1
 4
 0

最も多くの学生を受け入れた学科を探します。

In [48]:
maximum(resp_num)

12

In [49]:
for max_resp in 1:length(resp_prefs)
    if !(resp_num[max_resp] == maximum(resp_num))
        max_resp += 1
    else
        print(max_resp)
        break
    end
end  

122

最も多くの学生を受け入れた学科122と学生とのマッチングに着目します。

学科122に内定した学生が、学科１を何番目に志望していたかを、以下の関数を使ってみてみます。

In [50]:
function student_pref_rank(resp_number)
    x = 1
    while x <= num_students
        if x in resp_matched[indptr[resp_number]:indptr[resp_number+1]-1]
            pref_rank = 1
            x_pref = prop_prefs[x]
            while !(x_pref[pref_rank] == resp_number)
                pref_rank += 1
            end
            println("student $x; ""pref_rank = $pref_rank")
        end
        x += 1
    end
end

student_pref_rank (generic function with 1 method)

In [51]:
student_pref_rank(122)

student 26; pref_rank = 3
student 27; pref_rank = 4
student 49; pref_rank = 2
student 107; pref_rank = 2
student 144; pref_rank = 2
student 170; pref_rank = 3
student 171; pref_rank = 2
student 178; pref_rank = 7
student 194; pref_rank = 1
student 248; pref_rank = 1
student 273; pref_rank = 1
student 292; pref_rank = 2


今度は学科122に内定した学生が、学科122の受け入れ順位の何番目に位置していたかを、以下の関数を使ってみてみます。

In [56]:
function student_resp_rank(resp_number)
    w = resp_matched[indptr[resp_number]:indptr[resp_number+1]-1]
    for y in 1: caps[resp_number]
        if !(w[y] == 0) 
            resp_rank = 1
            pref = resp_prefs[resp_number]
            while !(pref[resp_rank] == w[y])
                resp_rank += 1
            end
            stu = w[y]
            println("student $stu; ""resp_rank = $resp_rank")
        else
            break
        end
    end
end

student_resp_rank (generic function with 1 method)

In [57]:
student_resp_rank(122)

student 194; resp_rank = 206
student 248; resp_rank = 95
student 273; resp_rank = 199
student 49; resp_rank = 210
student 107; resp_rank = 228
student 144; resp_rank = 285
student 292; resp_rank = 143
student 171; resp_rank = 286
student 178; resp_rank = 263
student 26; resp_rank = 34
student 27; resp_rank = 207
student 170; resp_rank = 104


ここから、学科122に内定した学生にとって、学科122は選好順位が比較的高く魅力的であった一方で、学科122に内定した学生は、学科122から見てさほど魅力的ではなかったと言えそうです。

以上で検証を終わります。